<a href="https://colab.research.google.com/github/HollyJY/202406_sumSchool_neuroAI/blob/main/W2_Spike_Network_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Tutorial: Cell-Type Specific Cortical Microcircuit

Author(s): Yifan Luo

Contact: yifan.luo@student.uva.nl

This notebook is based on the work of:

1) Potjans TC, Diesmann M. The cell-type specific cortical microcircuit: relating structure and activity in a full-scale spiking network model. Cereb Cortex. 2014 Mar;24(3):785-806. doi: 10.1093/cercor/bhs358. Epub 2012 Dec 2. PMID: 23203991; PMCID: PMC3920768.

2) Original model implementation: https://github.com/ModelDBRepository/244261

Please note that:

1) Every practical excercise has its solution attached right after it. Do not hesitate to check it out when you feel stuck in a certain step 😀;

2) We encourage you to discuss the To Think questions with your classmates and/or TA. Although skipping those question won't affect the execution of code.

**Recommended Hardware Accelerator**: CPU with high RAM (Colab Pro needed). If you are using local runtime, please ensure the **RAM is > 12 GB**.

Free Colab CPU runtime is an viable option as well. Although the simulation would be a bit slower.

**Estimated Time per Simulation (CPU runtime)**: ~22 min (initial simulation), ~9 min

**Estimated Time per Simulation (High RAM runtime)**: ~10 min (initial simulation), ~3 min

## 0. Preparations
In this module, we will create a folder in your google drive to store all the code and data for this tutorial.

In [ ]:
# make a 'spike_network' folder in your google drive to store the code and data
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('/content/drive/MyDrive/spike_network', exist_ok=True)

In [ ]:
# clone the github repository to spike_network
# note: this repository is modified based on the original model implementation
# for educational purposes of this tutorial
!git clone https://github.com/Beckinetic/SpikeNetwork.git '/content/drive/MyDrive/spike_network'

In [ ]:
# move the working directory to spike_network
%cd /content/drive/MyDrive/spike_network

# pull the changes to ensure the repo is up to date
!git pull

In [ ]:
# install required packages
!pip install -r requirements.txt
!pip install pdf2image
!apt-get install -y poppler-utils
print('Done')

In [ ]:
# import necessary libraries
import gc
import warnings

import matplotlib
matplotlib.use('Agg')  # Use 'Agg' backend for rendering plots in scripts or servers without GUI

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.gridspec as gridspec

import pandas as pd
import numpy as np
import scipy.stats as sc

from brian2 import *

warnings.filterwarnings('ignore')  # We are reckless!

# brian2 is the most important package we are going to use. It is dedicated to
# simulate spiking neural networks in python.

## 1. Building the Network
Now we are all set with prerequisites and ready to simulate some networks! First, let's take a look at the illustration of the network we are going to simulate.

<center><img src="https://raw.githubusercontent.com/Beckinetic/SpikeNetwork/master/images/network_structure.gif" width=450></center>

*Note.* Excitatory neurons (triangles) and inhibitory neurons (circles) are shown. Excitatory connections (black) and inhibitory connections (grey) with >.04 connectivity probability are shown.

This network corresponds to a biological cortical network of the surface area of 1 mm$^2$. The network simulates 8 neuron populations: L2/3e (excitatory neurons in layer 2/3), L2/3i (inhibitory neurons in layer 2/3), L4e, L4i, L5e, L5i, L6e, L6i, and Th (thalamic neurons). The neurons in this network are connected with other neurons within the same layer or from other layers. This network also should be able to receive various forms external inputs so we can study the spontaneous and stimulus-evoked activities.

To build this cortical model, we have a rough three-step plan:

1. Define the single neuron model (LIF)
2. Wire up the neurons to form a network
3. Define the external input forms

Let's get started with the LIF model!

Our first step is to determine the model for every single neuron in this network. In this case, we adopt the **leaky integrate-and-fire model** (LIF) as the neuron model. To put it simply, in this model the neuron may receive inputs (currents) from synapses and external sources, and if the inputs are frequent enough to cause the membrane potential to reach a certain threshold, the neuron may fire a spike. A more detailed introduction could be found [here](https://neuronaldynamics.epfl.ch/online/Ch1.S3.html).

<font color='red'>**To Think 1-1**</font>: Do you think the LIF model as the model for single neurons in spiking network serves the right level of model simplification? (Hint: compare it with Hodgkin-Huxley neurons and convolutional neural network neurons)

<center><img src="https://raw.githubusercontent.com/jeshraghian/snntorch/master/docs/_static/img/examples/tutorial2/2_1_neuronmodels.png"></center>

### LIF Neurons

The core bit of the LIF model is the input integrating equation (linear differential equation of membrane voltage):

$${dv}/{dt} = (-v+v_{r})/{\tau_m} + (I+I_{ext})/{C_m}$$

This equation simulates the membrane voltage changing behavior, including how the voltage decays to resting potential and how it changes when there's external inputs. To break it down:
- $v$ is the membrane potential of the neuron.
- $(-v + v_r) / \tau_m$ : This term describes how the membrane potential decays to the reset potential $v_r$ over time, governed by the membrane time constant $\tau_m$, a.k.a. the leaky integrator.
- $(I + I_{ext}) / C_m$ : This term adds the effects of input currents: $I$ is the synaptic current received by the neuron. $I_{ext}$ is an external current applied to the neuron.

You can read more about the derivation of this equation [here](https://neuronaldynamics.epfl.ch/online/Ch1.S3.html).

Now we need to transform this equation into something interpretable to Python. Thanks to brian2, this process is very much simplified. We first assign values to several LIF parameters:

In [ ]:
def LIFparams():
    tau_m = 10.0 * ms  # membrane time constant
    tau_ref = 2.0 * ms  # absolute refractory period
    Cm = 250.0 * pF  # membrane capacity
    v_r = -65.0 * mV  # reset potential
    v_th = -50.0 * mV  # fixed firing threshold
    return tau_m, tau_ref, Cm, v_r, v_th

# Note. Using brian2, you can add SI units to the variables

Now let's try write down the linear differential equation we've just seen with brian2. Note that you need to add the unit of the variable defined by the equation (in this case, volt) after a colon. And also in this particular case, you need to append `(unless refractory)` to the end of the equation. You may read more about brian2 usage in general [here](https://brian2.readthedocs.io/en/stable/resources/tutorials/1-intro-to-brian-neurons.html).

<font color="green">**To Do 1-1**</font> Replace `###YOUR CODE###` with the equation. Remember to remove `raise NotImplementedError` after you inserted your code.

In [ ]:
# Leaky integrate-and-fire model equations
# I: synaptic current
# tau_syn: synaptic time constant
# Iext: external current
LIFmodel = '''
	###YOUR CODE###
	dI/dt = -I/tau_syn : amp
	Iext : amp
	'''
# Reset condition
resetLIF = '''
	v = v_r
	'''

raise NotImplementedError

In [ ]:
#@title Solution To Do 1-1

# Leaky integrate-and-fire model equations
# I: synaptic current
# tau_syn: synaptic time constant
# Iext: external current
LIFmodel = '''
	dv/dt = (-v + v_r)/tau_m + (I+Iext)/Cm : volt (unless refractory)
	dI/dt = -I/tau_syn : amp
	Iext : amp
	'''
# Reset condition
resetLIF = '''
	v = v_r
	'''

So far we have defined the core passive membrane behavior pattern of a LIF neuron, and also how it handles external inputs. The firing threshold and refractory period length are fixed in our equation. That's basically evertything we need for a single LIF neuron. We just need to set up some initial parameter values when we actually run it.

Now let's proceed to connecting those neurons.



### Balanced Random Network Model

The balanced random network model is what we are using to characterize the neuronal population dynamics. It receives external inputs (i.e., background inputs) and propagate the stimulation to the whole network through synapses. Previous studies provided valuable data of the population size and the connection probabilities, and we can build the network based on it. Let's take a look and run the network parameters.


> Connection probability: the probability that a neuron in the presynaptic population forms at least 1 synapse with a neuron in the postsynaptic population.



In [ ]:
###############################################################################
# Network parameters
###############################################################################
# Population size per layer
#          2/3e   2/3i   4e    4i    5e    5i    6e     6i    Th
n_layer = [20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948, 902]

# Total cortical Population
N = sum(n_layer[:-1])

# Number of neurons accumulated
nn_cum = [0]
nn_cum.extend(cumsum(n_layer))

# Prob. connection table
#          From 2/3e    2/3i   4e     4i     5e     5i      6e     6i      Th        To
table = array([[0.101,  0.169, 0.044, 0.082, 0.032, 0.,     0.008, 0.,     0.    ], #2/3e
               [0.135,  0.137, 0.032, 0.052, 0.075, 0.,     0.004, 0.,     0.    ], #2/3i
               [0.008,  0.006, 0.050, 0.135, 0.007, 0.0003, 0.045, 0.,     0.0983], #4e
               [0.069,  0.003, 0.079, 0.160, 0.003, 0.,     0.106, 0.,     0.0619], #4i
               [0.100,  0.062, 0.051, 0.006, 0.083, 0.373,  0.020, 0.,     0.    ], #5e
               [0.055,  0.027, 0.026, 0.002, 0.060, 0.316,  0.009, 0.,     0.    ], #5i
               [0.016,  0.007, 0.021, 0.017, 0.057, 0.020,  0.040, 0.225,  0.0512], #6e
               [0.036,  0.001, 0.003, 0.001, 0.028, 0.008,  0.066, 0.144,  0.0196]])#6i

# Synapses parameters
d_ex = 1.5*ms      	# Excitatory delay
std_d_ex = 0.75*ms 	# Std. Excitatory delay
d_in = 0.80*ms      # Inhibitory delay
std_d_in = 0.4*ms  	# Std. Inhibitory delay
tau_syn = 0.5*ms    # Post-synaptic current time constant

# Layer-specific background input
bg_layer_specific = array([1600, 1500 ,2100, 1900, 2000, 1900, 2900, 2100])

# Layer-independent background input
bg_layer_independent = array([2000, 1850 ,2000, 1850, 2000, 1850, 2000, 1850])

We can see that the population size per layer, the connection probabilities, and the synapse parameters are already filled out. The connection table shows there might be recurrent intralayer connections (e.g., L2/3e -> L2/3e) and interlayer connections (e.g., L2/3e -> L4e). Note that every column shows the connections *from* the same presynaptic neurons, and every row shows the connection *to* the same postsynaptic neurons (the thalamic neurons can only serve as presynaptic neurons!).

You may also notice that there are layer-specific background inputs, which are external inputs to each layer and the numbers are estimated through experimental research. There are also layer-independent inputs, which have the identical number for every layer. The layer-independent inputs are used to test the robustness of this network. Actually, you may define the input array in various way to simulate different external input patterns. We'll talk about it later on in the robustness tests.

<font color="red">**To Think 1-2**</font>: The population size per layer is determined by anatomical studies. What is the approximate proportion of exitatory cells to inhibitory cells? What do you predict to happen in the network if we assign the same weight to synapses to inhibitory cells and synapses to excitatory cells?

With the population sizes, connection probabilities, and the synaptic data, we can wire up the neurons. In a balanced random network, every neuronal population forms connections to every other population, and the exact number of synapses in the connections is determined by the connection probability and population size. Once has the synapse quantity settled, the synapases are randomly distributed to the neurons in the population.

Now we have the population sizes and the connection probabilities, then we can calculate the number of synapses by the following equation:

$$
C_a = 1 - (1 - \frac{1}{N^{pre}N^{post}})^K
$$

$C_a$ is the connection probability, $N^{pre}$ and $N^{post}$ denote the presynaptic and postsynaptic neuron numbers, and $K$ is the number of the synapses.

With a bit transformation, we obtain:

$$
K = \ln(C_a) / \ln(1 - (1 - \frac{1}{N^{pre}N^{post}}))
$$

We can also consider using the Taylor series approximation of the original equation:

$$
C_a = \frac{K}{N^{pre}N^{post}} ⇒ K = \frac{C_a}{N^{pre} N^{post}}
$$

Much simpler! Later, we can test those two different ways to calculating synapse numbers and see if it affects the network.

<font color="red">**To Think 1-3**</font>: Try to derive the connection probability equation (Hint: think about the definition of connection probability).


Solution To Think 1-3

<details>
  <summary>Click to expand</summary>
We know that the connection probability is "the probability that a neuron in the presynaptic population forms at least 1 synapse with a neuron in the postsynaptic population". In other words, it is one minus the probability that a neuron in the presynaptic population forms no synapse to a neuron in the postsynaptic population.

$$C_a = 1 - P(\text{no synapse})$$

There are $N^{pre}N^{post}$ forms of synapses connecting different neurons. Imagine we are distributing K synapses to those niches, the probability of a particular synapse connecting two particular neurons will be $$\frac{1}{N^{pre}N^{post}}$$

And the opposite of that is this particular synapse does not connect those two neurons. The probability of this happening is $$1 - \frac{1}{N^{pre}N^{post}}$$

Then, the probability of none of the $K$ synapses connecting those two neurons, namely the probability that a neuron in the presynaptic population forms no synapse to a neuron in the postsynaptic population, would be $$(1 - \frac{1}{N^{pre}N^{post}})^K$$ We can substitute $P(\text{no synapse})$ with this, and finally we have

$$C_a = 1 - (1 - \frac{1}{N^{pre}N^{post}})^K$$
</details>

Once we have number of the synapses between two populations, we can randomly assign the synapses to the neurons. It's time for some Python coding!

<font color="green">**To Do 1-2**</font>: Replace `###YOUR CODE###` with the connection probability-synapse number equation(s). We use variable `nsyn` to denote the number of the synapses. Be aware: the number of synapses must be integers.

Remember to remove `raise NotImplementedError` after you inserted your code.

In [ ]:
def PDnet(NeuronGroup, stim, bg_type, w_ex, g, bg_freq, nsyn_type, thal):
    # P and D are the initials of the authors of the paper

    w_ex = w_ex*pA		   	# excitatory synaptic weight
    std_w_ex = 0.1*w_ex     # standard deviation weight

    pop = [] # Stores NeuronGroups, one for each population
    for r in range(0, 8):
        pop.append(NeuronGroup[nn_cum[r]:nn_cum[r+1]])

    ###########################################################################
    # Creating synapse connections
    ###########################################################################

    syn_model = '''
                w:amp			# synaptic weight
                '''

    # equations executed only when presynaptic spike occurs:
    # for excitatory connections
    pre_eq = '''
            I_post += w
            '''

    con = [] # Stores connections

    ###########################################################################
    # Connecting neurons
    ###########################################################################
    pre_index = [] # presynaptic neuron index
    post_index = [] # postsynaptic neuron index

    # c: column (presynaptic neurons)
    # r: row (postsynaptic neurons)
    for c in range(0, 8):
        for r in range(0, 8):

            if (nsyn_type==0):
                # number of synapses calculated with the original equation
                ###YOUR CODE###
                raise NotImplementedError
            elif (nsyn_type==1):
                # number of synapses calculated with the Taylor series approximation
                ###YOUR CODE###
                raise NotImplementedError
            pre_index = randint(n_layer[c], size=nsyn)
            post_index = randint(n_layer[r], size=nsyn)

            # Assign weights to the synapses
            if nsyn<1:
                pass
            else:
                # Excitatory connections
                if (c % 2) == 0:
                    # Synaptic weight from L4e to L2/3e is doubled as the data basis for these connections is not fully conclusive
                    if c == 2 and r == 0:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = '2.0*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    else:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = 'clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_ex + std_d_ex*randn(), 0.1*ms, d_ex*inf)'

                # Inhibitory connections
                else:
                    con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                    con[-1].connect(i = pre_index, j = post_index)
                    con[-1].w = '-g*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_in + std_d_in*randn(), 0.1*ms, d_in*inf)'

In [ ]:
#@title Solution To Do 1-2
def PDnet(NeuronGroup, stim, bg_type, w_ex, g, bg_freq, nsyn_type, thal):

    w_ex = w_ex*pA		   	# excitatory synaptic weight
    std_w_ex = 0.1*w_ex     # standard deviation weight

    pop = [] # Stores NeuronGroups, one for each population
    for r in range(0, 8):
        pop.append(NeuronGroup[nn_cum[r]:nn_cum[r+1]])

    ###########################################################################
    # Creating synapse connections
    ###########################################################################

    syn_model = '''
                w:amp			# synaptic weight
                '''

    # equations executed only when presynaptic spike occurs:
    # for excitatory connections
    pre_eq = '''
            I_post += w
            '''

    con = [] # Stores connections

    ###########################################################################
    # Connecting neurons
    ###########################################################################
    pre_index = [] # presynaptic neuron index
    post_index = [] # postsynaptic neuron index

    # c: column (presynaptic neurons)
    # r: row (postsynaptic neurons)
    for c in range(0, 8):
        for r in range(0, 8):

            if (nsyn_type==0):
                # number of synapses calculated with the original equation
                nsyn = int(log(1.0-table[r][c])/log(1.0 - (1.0/float(n_layer[c]*n_layer[r]))))
            elif (nsyn_type==1):
                # number of synapses calculated with the Taylor series approximation
                nsyn = int(n_layer[c]*n_layer[r]*table[r][c])
            pre_index = randint(n_layer[c], size=nsyn)
            post_index = randint(n_layer[r], size=nsyn)

            # assign weights to the synapses
            if nsyn<1:
                pass
            else:
                # Excitatory connections
                if (c % 2) == 0:
                    # Synaptic weight from L4e to L2/3e is doubled
                    if c == 2 and r == 0:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = '2.0*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    else:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = 'clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_ex + std_d_ex*randn(), 0.1*ms, d_ex*inf)'

                # Inhibitory connections
                else:
                    con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                    con[-1].connect(i = pre_index, j = post_index)
                    con[-1].w = '-g*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_in + std_d_in*randn(), 0.1*ms, d_in*inf)'

<font color="red">**To Think 1-4**</font>: You may noticed that every synapse has been assigned with a weight. The assignments to excitatory connections and inhibitory connections are a bit different:
```
#Excitatory connections
...
con[-1].w = 'clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
...
# Inhibitory connections
con[-1].w = '-g*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
```

The variable `g` is called inhibitory weight balance, and its default value in this tutorial is 4. Does the value remind you of something about the neuronal population size? Why do we balance the inhibitory weight?

The connections within the network are created, and there's one more thing we need to do: handling external inputs.

```
    ###########################################################################
    # Setting background input values
    ###########################################################################
    
    # Background number per layer
    if bg_type == 0:
        # layer-specific:
        bg_layer = bg_layer_specific
    elif bg_type == 1:
        # layer-independent:
        bg_layer = bg_layer_independent
```
We have defined two types of background inputs
1. layer-specific: external inputs from grey matter, white matter, and thalamic (independent from the thalamic inputs we defined in the parameters) sources. The number is determined by empirical research.
2. layer-independent: also external inputs from grey and white matter. The number is set as identical for each layer.

You can look up the network parameters for the layer-specific and layer-independent numbers.

The reason for layer-independent is that the data basis for layer-specific inputs is actually limited. To rigorously test this network, we need to investigate other input parameterization's effects. We can also create other custom background types to do more testing.

```
    ###########################################################################
    # Creating poissonian and DC-current background inputs
    ###########################################################################
    bg_in  = []
    if (stim==0):
        for r in range(0, 8):
            bg_in.append(PoissonInput(pop[r], 'I', bg_layer[r], bg_freq*Hz, weight=w_ex))
    
    # DC-current normalized by population
    if (stim == 1):
        for r in range(0, 8):
            NeuronGroup.Iext[pop[r]] = 0.3512*pA*bg_layer[r]
```
We also model two types inputs: poissonian and DC-current background inputs:

> Poissonian input: a type of stochastic or random input where the timing of incoming spikes follows a poisson process. This means the spikes arrive randomly but with a certain average rate. Poissonian input is used to simulate background synaptic noise.

> DC (direct current) input: a constant or fixed input current applied to the neuron. Unlike Poissonian input, it does not vary over time. This input is analogous to injecting a constant electrical current into a neuron. DC input is often used to simulate a constant external stimulus.

We can test these two types of inputs in the later robustness tests.

```
    ###############################################################################
    # Creating thalamic neurons as poissonian inputs
    ###############################################################################
    thal_con = []
    thal_input = []
    if thal=="ON":
        thal_input = PoissonGroup(n_layer[8], rates=120.0*Hz)	#from PD paper: rates=15Hz
        for r in range(0,8):
            thal_con.append(Synapses(thal_input, pop[r], model=syn_model, on_pre=pre_eq))
            thal_con[-1].connect(p=table[r][8])
            thal_con[-1].w = 0.0
```

To simulate stimulus-evoked activity, we explicitly model the thalamic input to L4 and L6 by a thalamic population of 902 neurons (Peters and Payne 1993). You can find its own neuronal population and connection probabilities in the network parameters section. By defining the thalamic input this way, we can fine-tune the thalamic inputs towards specific neuronal populations.



Phew, we finally have everything for the network building! Now we just need to assemble the modules to create the complete network function. We also need to add a spike monitor to the network to collect the spiking data.

<font color="green">**To Do 1-3**</font> (1) Copy and paste the modules to assemble the `PDNet` (2) Create a spike monitor called `smon_net` using brian2 at the end of the function. You can refer to the [brian2 tutorial](https://brian2.readthedocs.io/en/stable/resources/tutorials/1-intro-to-brian-neurons.html) about how to do it.

In [ ]:
def PDnet(NeuronGroup, stim, bg_type, w_ex, g, bg_freq, nsyn_type, thal):
    # P and D are the initials of the authors of the paper

    w_ex = w_ex*pA		   	# excitatory synaptic weight
    std_w_ex = 0.1*w_ex     # standard deviation weight

    pop = [] # Stores NeuronGroups, one for each population
    for r in range(0, 8):
        pop.append(NeuronGroup[nn_cum[r]:nn_cum[r+1]])

    ###########################################################################
    # Creating synapse connections
    ###########################################################################

    syn_model = '''
                w:amp			# synaptic weight
                '''

    # equations executed only when presynaptic spike occurs:
    # for excitatory connections
    pre_eq = '''
            I_post += w
            '''

    con = [] # Stores connections

    ###########################################################################
    # Connecting neurons
    ###########################################################################

    raise NotImplementedError

    ###########################################################################
    # Setting background input values
    ###########################################################################

    # Background number per layer
    if bg_type == 0:
        # layer-specific:
        bg_layer = bg_layer_specific
    elif bg_type == 1:
        # layer-independent:
        bg_layer = bg_layer_independent

    ###########################################################################
    # Creating poissonian and DC-current background inputs
    ###########################################################################
    bg_in  = []
    # poissonian inputs
    if (stim==0):
        for r in range(0, 8):
            bg_in.append(PoissonInput(pop[r], 'I', bg_layer[r], bg_freq*Hz, weight=w_ex))

    # DC-current normalized by population
    if (stim == 1):
        for r in range(0, 8):
            NeuronGroup.Iext[pop[r]] = 0.3512*pA*bg_layer[r]

    ###########################################################################
    # Creating thalamic neurons as poissonian inputs
    ###########################################################################
    thal_con = []
    thal_input = []
    if thal=="ON":
        thal_input = PoissonGroup(n_layer[8], rates=120.0*Hz)    #from PD paper: rates=15Hz
        for r in range(0,8):
            thal_con.append(Synapses(thal_input, pop[r], model=syn_model, on_pre=pre_eq))
            thal_con[-1].connect(p=table[r][8])
            thal_con[-1].w = 0.0

    ###########################################################################
    # Creating spike monitors
    ###########################################################################
    ##YOUR CODE##
    raise NotImplementedError

    return pop, con, bg_in, smon_net, thal_input ,thal_con

In [ ]:
#@title Solution To Do 1-3

def PDnet(NeuronGroup, stim, bg_type, w_ex, g, bg_freq, nsyn_type, thal):
    # P and D are the initials of the authors of the paper

    w_ex = w_ex*pA		   	# excitatory synaptic weight
    std_w_ex = 0.1*w_ex     # standard deviation weight

    pop = [] # Stores NeuronGroups, one for each population
    for r in range(0, 8):
        pop.append(NeuronGroup[nn_cum[r]:nn_cum[r+1]])

    ###########################################################################
    # Creating synapse connections
    ###########################################################################

    syn_model = '''
                w:amp			# synaptic weight
                '''

    # equations executed only when presynaptic spike occurs:
    # for excitatory connections
    pre_eq = '''
            I_post += w
            '''

    con = [] # Stores connections

    ###########################################################################
    # Connecting neurons
    ###########################################################################

    pre_index = [] # presynaptic neuron index
    post_index = [] # postsynaptic neuron index

    # c: column (presynaptic neurons)
    # r: row (postsynaptic neurons)
    for c in range(0, 8):
        for r in range(0, 8):

            if (nsyn_type==0):
                # number of synapses calculated with the original equation
                nsyn = int(log(1.0-table[r][c])/log(1.0 - (1.0/float(n_layer[c]*n_layer[r]))))
            elif (nsyn_type==1):
                # number of synapses calculated with the Taylor series approximation
                nsyn = int(n_layer[c]*n_layer[r]*table[r][c])
            pre_index = randint(n_layer[c], size=nsyn)
            post_index = randint(n_layer[r], size=nsyn)

            # assign weights to the synapses
            if nsyn<1:
                pass
            else:
                # Excitatory connections
                if (c % 2) == 0:
                    # Synaptic weight from L4e to L2/3e is doubled
                    if c == 2 and r == 0:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = '2.0*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    else:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = 'clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_ex + std_d_ex*randn(), 0.1*ms, d_ex*inf)'

                # Inhibitory connections
                else:
                    con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                    con[-1].connect(i = pre_index, j = post_index)
                    con[-1].w = '-g*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_in + std_d_in*randn(), 0.1*ms, d_in*inf)'

    ###########################################################################
    # Setting background input values
    ###########################################################################

    # Background number per layer
    if bg_type == 0:
        # layer-specific:
        bg_layer = bg_layer_specific
    elif bg_type == 1:
        # layer-independent:
        bg_layer = bg_layer_independent

    ###########################################################################
    # Creating poissonian and DC-current background inputs
    ###########################################################################
    bg_in  = []
    if (stim==0):
        for r in range(0, 8):
            bg_in.append(PoissonInput(pop[r], 'I', bg_layer[r], bg_freq*Hz, weight=w_ex))

    # DC-current normalized by population
    if (stim == 1):
        for r in range(0, 8):
            NeuronGroup.Iext[pop[r]] = 0.3512*pA*bg_layer[r]

    ###########################################################################
    # Creating thalamic neurons as poissonian inputs
    ##########################################################################
    thal_con = []
    thal_input = []
    if thal=="ON":
        thal_input = PoissonGroup(n_layer[8], rates=120.0*Hz) #from PD paper: rates=15Hz
        for r in range(0,8):
            thal_con.append(Synapses(thal_input, pop[r], model=syn_model, on_pre=pre_eq))
            thal_con[-1].connect(p=table[r][8])
            thal_con[-1].w = 0.0

    ###########################################################################
    # Creating spike monitors
    ###########################################################################
    smon_net = SpikeMonitor(NeuronGroup)

    return pop, con, bg_in, smon_net, thal_input ,thal_con

For the simulation, we still need to create a `runParams` function, so we can decide how long the simulation persists, the background input type, whether turn on the thalamic inputs, etc. And we also need use this function to add the LIF model definition to the network.

In [ ]:
def runParams(tsim=1.0, bg_type=0, stim=0, w_ex=87.8, g=4.0, bg_freq=8.0, nsyn_type=0, thal='OFF', filename=None):

    ###########################################################################
    # Simulation parameters
    ###########################################################################
    defaultclock.dt = 0.1*ms    # timestep of numerical integration method

    # neuron model
    eqs = LIFmodel
    reset = resetLIF
    tau_m, tau_ref, Cm, v_r, v_th = LIFparams()

    ###########################################################################
    # Creating neurons
    ###########################################################################
    neurons = NeuronGroup(N, eqs, threshold='v>v_th', reset=reset, \
                            method='linear', refractory=tau_ref)

    # seting initial values for membrane potential and currents
    neurons.v = '-58.0*mV + 10.0*mV*randn()'
    neurons.I = 0.0*pA      # initial value for synaptic currents
    neurons.Iext = 0.0*pA   # constant external current

    pop, con, bg_in, smon_net, thal_input, thal_con = PDnet(neurons, stim, \
                                            bg_type, w_ex, g, bg_freq, nsyn_type, thal)

    ###########################################################################
    # Running the simulation
    ###########################################################################
    net = Network(collect())

    if (thal == 'OFF'):
        net.add(neurons,pop, con, bg_in)    # Adding objects to the simulation
        net.run(tsim*second, report='stdout')

    elif (thal == 'ON'):
        w_thal = w_ex*pA            # excitatory synaptic weight from thalamus
        std_w_thal = w_thal*0.1     # standard deviation weigth
        net.add(neurons,pop, con, bg_in, thal_input, thal_con)    # Adding objects to the simulation

        for repeat in range(0,int(tsim)):
            net.run(0.7*second,report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Adding thalamic input
            for r in range(0,8):
                thal_con[r].w = 'clip((w_thal + std_w_thal*randn()),w_thal*0.0, w_thal*inf)'
            net.run(0.01*second, report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Removing thalamic input
            for r in range(0,8):
                thal_con[r].w = 0
            net.run(0.29*second, report='stdout')
            gc.collect()    #garbage collector to clean memory

    ###########################################################################
    # Saving raster plot
    ###########################################################################
    savetxt(filename, c_[smon_net.i,smon_net.t/ms],fmt="%i %.2f")

## 2. Spontaneous Activity
### Simulation

Now we may begin the network simulation. We start with the spontaneous activity where there is only background Poissonian inputs.

In [ ]:
#@title Helper Functions: Simulation General
def filename(g, bgrate, suffix = ''):
    return '../data/data_raster_g' + str(g) + '_bgrate' + str(bgrate) +  suffix + '.dat'

# Function used in multiple runs: define different random seeds and clean memory
def runParamsParallel(s=1, g=4, bg_type=0, bg_freq=8.0, stim=0, tsim=1.0, filename=None):
    seed(s*1000)
    runParams(tsim=tsim, bg_type=bg_type, stim=stim, g=g, bg_freq=bg_freq, filename=filename)
    gc.collect()

# default seed of pseudo random numbers to test reproducibility
s = 1000
seed(s)

# choose serial = False to run multiple simulations in parallel
serial = True
num_cores = -1 # number of cores to run in parallel

*Note*. All the result data is stored in `/content/drive/MyDrive/spike_network/data`. And they are named using `g`, `bg`, and a custom suffix. Check the helper functions for naming rules.

In [ ]:
# run the spontaneous activity simulation

# it is recommended to simulate the network for 60 secs or longer, but since our
# class time is limited, we'll only simulate 1 sec, which may take ~2 min

# the initialization of the network may be slow due to unknown reasons,
# which may prolong the execution to ~10 min or even more. If you think it's
# going too slow, you can try run the next code cell as an alternative option
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 0 # Poissonian inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, filename=filename(g, bg, 'default'))
gc.collect() #garbage collector to clean memory

In [ ]:
#@title Alternative Simulation Option

# run the simulation!
%cd /content/drive/MyDrive/spike_network/code

# Note. It may take time longer than expected due to variance in initialization
!python netRun.py 0 1

### Raster Plot

We can make a raster plot to showcase the spiking of neurons in the simulation time. Before plotting, the spiking data needs to be sampled and sorted.

In [ ]:
###############################################################################
# Defining plotting parameters
###############################################################################

# cortical layer labels: e for excitatory; i for inhibitory
lname = ['L23e', 'L23i', 'L4e', 'L4i', 'L5e', 'L5i','L6e', 'L6i']

# number of neurons by layer
n_layer_plot = [0, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948];
l_bins = np.cumsum(n_layer_plot) # cumulative number of neurons by layer
N = np.sum(n_layer_plot)         # total number of neurons

# sampling parameters
psample = 0.025 # percentage of neurons by layer for the raster plot
n_sample = 1000 # number of neurons by layer for sampled measures

In [ ]:
def preprocess(tsim, filename, lname, n_layer_plot, l_bins, N, psample, n_sample):
  #############################################################################
  # Loading data and defining parameters
  #############################################################################
  data = pd.read_csv(filename, sep=" ",
                      header=None, names=['i','t'])

  # grouping spiking times for each neuron
  keys,values = data.sort_values(['i','t']).values.T
  ukeys,index=np.unique(keys,True)
  arrays=np.split(values,index[1:])

  spk_neuron = pd.DataFrame({'i':range(0,N),'t':[[]]*N})

  # DEPRECATED: List of arrays cannot be directly assigned to dataframe
  # spk_neuron.iloc[ukeys.astype(int),1] = arrays
  for neuron_index, spike_times in zip(ukeys.astype(int), arrays):
      spk_neuron.at[neuron_index, 't'] = spike_times.tolist()

  # creating a flag to identify cortical layers
  spk_neuron['layer'] = pd.cut(spk_neuron['i'], l_bins, labels=lname, right=False)
  data['layer'] = pd.cut(data['i'], l_bins, labels=lname, right=False)

  # sampling data:
  spk_neuron = spk_neuron.groupby(['layer'], observed=False).apply(lambda x: x.sample(n=n_sample))

  # reset the index to remove the ambiguity
  spk_neuron = spk_neuron.reset_index(drop=True)

  # measures DataFrame to store firing rates, synchrony, and irrgularity
  measures_layer = pd.DataFrame(index=lname)

  return spk_neuron, measures_layer, data

In [ ]:
%cd /content/drive/MyDrive/spike_network/code
spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, 'default'), lname, n_layer_plot, l_bins, N, psample, n_sample)

In [ ]:
%matplotlib inline

def rasterplot(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample):
  ###############################################################################
  # Raster plot
  ###############################################################################
  plt.figure(figsize=(7.5,10))
  plt.gca().set_yticklabels([])
  acum_index = 0

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  for i in range(len(lname)):
      index_start = l_bins[i]
      index_end = l_bins[i]+int(psample*n_layer_plot[i+1])

      x = data.t[data.i.isin(range(index_start,index_end))]
      y = data.i[data.i.isin(range(index_start,index_end))] + acum_index - index_start

      plt.plot(x/1000.0,y,'.',markersize=dotsize,color=dotcolor[i])

      # layers labels
      xpos = tsim-440/1000.0
      ypos = acum_index + (index_end-index_start)/2.0
      plt.text(xpos,ypos,lname[i],horizontalalignment='center', fontweight='bold')

      acum_index = acum_index + (index_end-index_start)

  plt.xlim(tsim-400/1000.0,tsim)
  plt.ylim(0,acum_index)
  plt.xlabel('time [s]')
  plt.ylabel(' ')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
rasterplot(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)

In this raster plot, we display the spiking per layer of 0.025 of the total neurons in the last 400 ms in our simulation.

<font color="red">**To Think 2-1**</font> What can you tell from this raster plot demonstrating the spontaneous activities of this networks? Does it align with [the figure in the original paper](https://academic.oup.com/view-large/figure/80937186/bhs35806.gif)?

The raster plot exhibits the different spiking behaviours across those layers. Other quantative measures are also quite informative. We will plot the firing rate, irrgularity, and synchrony of each neuron population.
> **Firing Rate**: the number of spikes a neuron generates per unit of time

> **Irregularity**: single-unit spike trains quantified by the coefficient of variation of the interspike intervals.

> **Synchrony**: multiunit spiking activity quantified by the variance of the spike count histogram (bin width 3 ms) divided by its mean (i.e., the Fano factor).

In [ ]:
%matplotlib inline

def firing_rate(spk_neuron, measures_layer, tsim):
  #############################################################################
  # Firing rates
  #############################################################################
  plt.figure(figsize=(7.5,10))

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  freq = []
  freq = [float(len(spk_neuron.t[i]))/tsim for i in range(len(spk_neuron))]
  spk_neuron['f'] = freq

  measures_layer['f'] = spk_neuron.groupby(['layer'], observed=False)['f'].mean()

  # boxplot of firing rates by layer
  bplot = spk_neuron.boxplot(column = 'f', by = 'layer', showmeans=True,
                      vert = False, rot = 30, patch_artist=True, sym='+',
                      return_type='dict', grid=False)

  [bplot[0]['boxes'][i].set_facecolor(dotcolor[i]) for i in range(0,len(bplot[0]['boxes']))]
  [bplot[0]['means'][i].set_markerfacecolor('white') for i in range(0,len(bplot[0]['boxes']))]
  [bplot[0]['means'][i].set_markeredgecolor('k') for i in range(0,len(bplot[0]['boxes']))]

  plt.title("")
  plt.ylabel("")
  plt.xlabel('firing rates[Hz]')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
firing_rate(spk_neuron, measures_layer, tsim)

<font color="green">**To Do 2-1**</font>: Replace `##YOUR CODE##` with your code to complete the irregularity plotting function.

In [ ]:
%matplotlib inline

def irregularity(spk_neuron, measures_layer):
  #############################################################################
  # Interspike intervals + coefficient of variation
  #############################################################################
  plt.figure(figsize=(7.5,10))

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  # interspike intervals
  isi = []
  isi = ##YOUR CODE##
  raise NotImplementedError()

  # coefficient of variation
  cv = []
  cv = ##YOUR CODE##
  raise NotImplementedError()

  spk_neuron['cv'] = cv

  measures_layer['cv'] = spk_neuron.groupby(['layer'], observed=False)['cv'].mean()

  # barplot of mean CV
  plt.subplot2grid((3,2),(1,1))
  measures_layer['cv'].plot.barh(edgecolor='k' ,color=dotcolor, rot=30, width=0.8)
  plt.ylabel("")
  plt.xlabel('irregularity')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
#@title Solution To Do 2-1
%matplotlib inline

def irregularity(spk_neuron, measures_layer):
  #############################################################################
  # Interspike intervals + coefficient of variation
  #############################################################################
  plt.figure(figsize=(7.5,10))

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  # interspike intervals
  isi = []
  isi = [np.diff(spk_neuron.t[i]) for i in range(len(spk_neuron))]

  # coefficient of variation
  cv = []
  cv = [np.std(isi[i])/np.mean(isi[i]) if len(isi[i])>1 else np.nan\
          for i in range(len(spk_neuron))]
  spk_neuron['cv'] = cv

  measures_layer['cv'] = spk_neuron.groupby(['layer'], observed=False)['cv'].mean()

  # barplot of mean CV
  plt.subplot2grid((3,2),(1,1))
  measures_layer['cv'].plot.barh(edgecolor='k' ,color=dotcolor, rot=30, width=0.8)
  plt.ylabel("")
  plt.xlabel('irregularity')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
irregularity(spk_neuron, measures_layer)

<font color="green">**To Do 2-2**</font>: Replace `##YOUR CODE##` with your code to complete the synchrony plotting function.

In [ ]:
%matplotlib inline

def synchrony(spk_neuron, measures_layer):
  #############################################################################
  # Synchrony index
  #############################################################################
  plt.figure(figsize=(7.5,10))

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  sync = []
  # define 3 ms bin
  bins = np.arange(0,tsim*1000.0+3.0,3)

  for i in range(len(lname)):
    ##YOUR CODE##
    # 1. Filter out the spiking neurons of this layer from `spk_neuron`
    # 2. Use np.histogram to derive the spike counts of each bin from `data`
    # 3. Use spiking counts to calculate synchrony. Note that we calculate the
    #    sychrony based on the last 500 ms data, so use `count[166:]` (spiking
    #    counts data from 498 ms onwards).
    raise NotImplementedError()


  measures_layer['sync'] = sync

  # barplot of synchrony index
  y_pos = np.arange(len(lname))
  plt.subplot2grid((3,2),(2,1))
  measures_layer['sync'].plot.barh(edgecolor='k' ,color=dotcolor, rot=30, width=0.8)
  plt.ylabel("")
  plt.xlabel('synchrony')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
#@title Solution To Do 2-2
%matplotlib inline

def synchrony(spk_neuron, measures_layer):
  #############################################################################
  # Synchrony index
  #############################################################################
  plt.figure(figsize=(7.5,10))

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  sync = []
  bins = np.arange(0,tsim*1000.0+3.0,3)

  for i in range(len(lname)):
      index_sample = spk_neuron.i[spk_neuron.layer.isin([lname[i]])]
      count, division = np.histogram(data.t[data.i.isin(index_sample)],bins=bins)
      sync.append(np.var(count[166:])/np.mean(count[166:]))

  measures_layer['sync'] = sync

  # barplot of synchrony index
  y_pos = np.arange(len(lname))
  plt.subplot2grid((3,2),(2,1))
  measures_layer['sync'].plot.barh(edgecolor='k' ,color=dotcolor, rot=30, width=0.8)
  plt.ylabel("")
  plt.xlabel('synchrony')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
synchrony(spk_neuron, measures_layer)

We may also plot the histogram to inspect the spiking distribution along the time axis.

In [ ]:
%matplotlib inline

def spike_histograms(spk_neuron, lname, tsim):
    #############################################################################
    # Spike count histograms for each layer
    #############################################################################
    n_layers = len(lname)

    fig, axes = plt.subplots(n_layers, 1, figsize=(7.5, 10))  # One subplot per layer

    bins = np.arange(0, tsim * 1000.0 + 3.0, 3)

    for i in range(len(lname)):
      index_sample = spk_neuron.i[spk_neuron.layer.isin([lname[i]])]
      count, division = np.histogram(data.t[data.i.isin(index_sample)],bins=bins)

      ax = axes[i]
      ax.bar(bins[167:], count[166:], width=3, color='blue', edgecolor='k')

      ax.set_title(lname[i])
      ax.set_xlabel("time")
      ax.set_ylabel("spike count")

    plt.tight_layout()
    plt.show()

In [ ]:
spike_histograms(spk_neuron, lname, tsim)

<font color="red">**To Think 2-2**</font>
1. Do the results in alignment with [the stimulation results in the paper](https://academic.oup.com/view-large/figure/80937186/bhs35806.gif)?
2. Are our simulated firing rates in agreement with [the experimental results](https://academic.oup.com/view-large/80937194)?
3. Compare firing rates of excitatory and inhibitory cells of different layers, which one is higher? What could be the neuroscience implications of it?
4. Does the network in general show asychronous irregular (AI) firing behaviour? Consider the irregularity and synchrony data, and compare the raster plot with the following reference.

<img src="https://raw.githubusercontent.com/Beckinetic/SpikeNetwork/master/images/behav_reference.jpg" width=450>

(Brunel, 2000)

Great! We now have finished every necessary component of this simulation workflow. Now it's time to do more testing on this network.

## 3. Network Robustness Test: Background Input Forms
In the network building section we defined different forms of inputs. We are going to replace the layer-specific Poissonian inputs with layer-independent Poissonian inputs (identical input "intensity" to every layer), and with layer-specific DC inputs. We are also going to inspect the results with the measures we defined before.

### Layer-independent Poissonian Inputs

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 1 # layer-independent inputs
stim = 0 # Poissonian inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, filename=filename(g, bg, '_layer-independent'))
gc.collect() #garbage collector to clean memory

In [ ]:
#@title Alternative Simulation Option
# run the simulation!
%cd /content/drive/MyDrive/spike_network/code

# Note. It may take time longer than expected due to variance in initialization
!python netRun.py 1 0

In [ ]:
spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, '_layer-independent'), lname, n_layer_plot, l_bins, N, psample, n_sample)
firing_rate(spk_neuron, measures_layer, tsim)
irregularity(spk_neuron, measures_layer)
synchrony(spk_neuron, measures_layer)

### Layer-specific DC Inputs

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 1 # DC current inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, filename=filename(g, bg, '_DC'))
gc.collect() #garbage collector to clean memory

In [ ]:
#@title Alternative Simulation Option
# run the simulation!
%cd /content/drive/MyDrive/spike_network/code

# Note. It may take time longer than expected due to variance in initialization
!python netRun.py 0 1

In [ ]:
spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, '_DC'), lname, n_layer_plot, l_bins, N, psample, n_sample)
firing_rate(spk_neuron, measures_layer, tsim)
irregularity(spk_neuron, measures_layer)
synchrony(spk_neuron, measures_layer)

<font color="red">**To Think 3-1**</font>: Compare the results with the spontaneous activity simulation (layer-specific Poissonian inputs) results.
1. What changes and what features are conserved? Do you find the network robust to the input forms? What could be reasons for it to be/not to be robust?
2. What happened to L6e when the inputs become layer-independent (you may also make the raster plot)? Does it give you some implications on the biological realism of the input forms?

## 4. Stimulus Evoked Activity

To simulate stimulus-evoked activity, we explicitly model the thalamic input to L4 and L6 by a thalamic population of 902 neurons (Peters and Payne 1993). The thalamic stimulus will be injected from 700 ms to 710 ms.

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 1 # DC inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, thal="ON", filename=filename(g, bg, '_thal'))
gc.collect() #garbage collector to clean memory

In [ ]:
#@title Alternative Simulation Option
# run the simulation!
%cd /content/drive/MyDrive/spike_network/code

# Note. It may take time longer than expected due to variance in initialization
!python netRun.py 5 1

We make a slightly modified version of raster plot function to zoom into the moment that the thalamic stimulus transpired.

In [ ]:
%matplotlib inline

def rasterplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample):
  ###############################################################################
  # Raster plot
  ###############################################################################
  plt.figure(figsize=(7.5,10))
  plt.gca().set_yticklabels([])
  acum_index = 0

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  for i in range(len(lname)):
      index_start = l_bins[i]
      index_end = l_bins[i]+int(psample*n_layer_plot[i+1])

      x = data.t[data.i.isin(range(index_start,index_end))]
      y = data.i[data.i.isin(range(index_start,index_end))] + acum_index - index_start

      plt.plot(x,y,'.',markersize=dotsize,color=dotcolor[i])

      # layers labels
      xpos = tsim*1000.0-312
      ypos = acum_index + (index_end-index_start)/2.0
      plt.text(xpos,ypos,lname[i],horizontalalignment='center', fontsize=12, rotation=30)


      acum_index = acum_index + (index_end-index_start)

  plt.xlim(tsim*1000.0-310,tsim*1000.0-290)
  plt.ylim(0,acum_index)
  plt.xlabel('time [ms]')
  plt.ylabel(' ')
  plt.gca().invert_yaxis()
  plt.show()

In [ ]:
spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, '_thal'), lname, n_layer_plot, l_bins, N, psample, n_sample)
rasterplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)

Furthermore, we can make a step plot to quantify the spike times of each population during the thalamic input.

In [ ]:
%matplotlib inline

def stepplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample):
  spk_count = []
  bins = np.arange(0,tsim*1000.0+.5,.5)

  for i in range(len(lname)):
      index_start = l_bins[i]
      index_end = l_bins[i]+int(psample*n_layer_plot[i+1])
      count, division = np.histogram(data.t[data.i.isin(np.arange(index_start,index_end))],bins=bins)
      #count = sum(np.split(count,int(tsim)))/float(tsim)
      spk_count.append([count])

  measures_layer['spk_count'] = spk_count

  for i in range(0,len(lname),2):
      plt.subplot2grid((4,3),(int(i/2),1),colspan=2)
      plt.step(np.arange(-10,30,0.5),measures_layer.spk_count[i][0][1380:1460], label=lname[i])
      plt.step(np.arange(-10,30,0.5),measures_layer.spk_count[i+1][0][1380:1460], label=lname[i+1])
      plt.legend()
      if i != 6:
          ax = plt.gca()
          ax.xaxis.set_visible(False)
      else:
          ax = plt.gca()
          ax.yaxis.set_visible(True)
      plt.ylim(0,13)

  plt.show()

In [ ]:
stepplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)

<font color="red">**To Think 4-1**</font>

1. Compare the spiking with the spontaneous activities, what are the differences?
2. Try to sort out the spiking order of the neurons during the thalamic input. Compare it with the [flow map](https://academic.oup.com/view-large/figure/80937217/bhs35811.gif) in the paper. It's been argued that there is a feed-forward loop from L4 to L2/3 to L5. Do you concur with this view?
3. Why are some inhibitory neurons fired after the excitatory neurons' spiking?

<font color="green">**To Do 4-1**</font>: Modify the network parameters to remove the thalamic inputs to L6 and only preserve the input to L4. Run the simulation and make necessary plots to draw your conclusion.

In [ ]:
##YOUR CODE##

In [ ]:
#@title Solution To Do 4-1 (Code Only)

# Modify the prob. connection table
#          From 2/3e    2/3i   4e     4i     5e     5i      6e     6i      Th        To
table = array([[0.101,  0.169, 0.044, 0.082, 0.032, 0.,     0.008, 0.,     0.    ], #2/3e
               [0.135,  0.137, 0.032, 0.052, 0.075, 0.,     0.004, 0.,     0.    ], #2/3i
               [0.008,  0.006, 0.050, 0.135, 0.007, 0.0003, 0.045, 0.,     0.0983], #4e
               [0.069,  0.003, 0.079, 0.160, 0.003, 0.,     0.106, 0.,     0.0619], #4i
               [0.100,  0.062, 0.051, 0.006, 0.083, 0.373,  0.020, 0.,     0.    ], #5e
               [0.055,  0.027, 0.026, 0.002, 0.060, 0.316,  0.009, 0.,     0.    ], #5i
               [0.016,  0.007, 0.021, 0.017, 0.057, 0.020,  0.040, 0.225,  0.    ], #6e
               [0.036,  0.001, 0.003, 0.001, 0.028, 0.008,  0.066, 0.144,  0.    ]])#6i

%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 1 # DC inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, thal="ON", filename=filename(g, bg, '_thal_L4'))
gc.collect() #garbage collector to clean memory

spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, '_thal_L4'), lname, n_layer_plot, l_bins, N, psample, n_sample)
rasterplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)
stepplot_thal(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)

## 5. Prefrontal Top-Down Stimulus

<font color="green">**To Do 4-1**</font>: Modify the network parameters, `PDnet`, `runParams` and other relevant components to simulate prefrontal top-down stimulus to this network. You may use the thalamic inputs as reference. Make raster plot and step plot to illustrate your results. Compare the results with previous thalamic stimulus results and spontaneous activity results.

Some parameters you may need:

**PFC Neuron Number**: 1000

**PFC Connection Probability**:
```
# Prob. connection table
#          From 2/3e    2/3i   4e     4i     5e     5i      6e     6i      Th      Pfc      To
table = array([[0.101,  0.169, 0.044, 0.082, 0.032, 0.,     0.008, 0.,     0.,     0.0900], #2/3e
               [0.135,  0.137, 0.032, 0.052, 0.075, 0.,     0.004, 0.,     0.,     0.0550], #2/3i
               [0.008,  0.006, 0.050, 0.135, 0.007, 0.0003, 0.045, 0.,     0.0983, 0.    ], #4e
               [0.069,  0.003, 0.079, 0.160, 0.003, 0.,     0.106, 0.,     0.0619, 0.    ], #4i
               [0.100,  0.062, 0.051, 0.006, 0.083, 0.373,  0.020, 0.,     0.,     0.0500], #5e
               [0.055,  0.027, 0.026, 0.002, 0.060, 0.316,  0.009, 0.,     0.,     0.0200], #5i
               [0.016,  0.007, 0.021, 0.017, 0.057, 0.020,  0.040, 0.225,  0.0512, 0.0500], #6e
               [0.036,  0.001, 0.003, 0.001, 0.028, 0.008,  0.066, 0.144,  0.0196, 0.0200]])#6i
```

**Rates**: 50Hz

**Stimulus Time**: 500ms ~ 800ms

In [ ]:
##YOUR CODE##

In [ ]:
#@title Solution To Do 5-1 (Code Only)

###############################################################################
# Network parameters
###############################################################################
# Population size per layer
#          2/3e   2/3i   4e    4i    5e    5i    6e     6i    Th   Pfc
n_layer = [20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948, 902, 1000]

# Total cortical Population
N = sum(n_layer[:-1])

# Number of neurons accumulated
nn_cum = [0]
nn_cum.extend(cumsum(n_layer))

# Prob. connection table
#          From 2/3e    2/3i   4e     4i     5e     5i      6e     6i      Th      Pfc      To
table = array([[0.101,  0.169, 0.044, 0.082, 0.032, 0.,     0.008, 0.,     0.,     0.0900], #2/3e
               [0.135,  0.137, 0.032, 0.052, 0.075, 0.,     0.004, 0.,     0.,     0.0550], #2/3i
               [0.008,  0.006, 0.050, 0.135, 0.007, 0.0003, 0.045, 0.,     0.0983, 0.    ], #4e
               [0.069,  0.003, 0.079, 0.160, 0.003, 0.,     0.106, 0.,     0.0619, 0.    ], #4i
               [0.100,  0.062, 0.051, 0.006, 0.083, 0.373,  0.020, 0.,     0.,     0.0500], #5e
               [0.055,  0.027, 0.026, 0.002, 0.060, 0.316,  0.009, 0.,     0.,     0.0200], #5i
               [0.016,  0.007, 0.021, 0.017, 0.057, 0.020,  0.040, 0.225,  0.0512, 0.0500], #6e
               [0.036,  0.001, 0.003, 0.001, 0.028, 0.008,  0.066, 0.144,  0.0196, 0.0200]])#6i

# Synapses parameters
d_ex = 1.5*ms      	# Excitatory delay
std_d_ex = 0.75*ms 	# Std. Excitatory delay
d_in = 0.80*ms      # Inhibitory delay
std_d_in = 0.4*ms  	# Std. Inhibitory delay
tau_syn = 0.5*ms    # Post-synaptic current time constant

# Layer-specific background input
bg_layer_specific = array([1600, 1500 ,2100, 1900, 2000, 1900, 2900, 2100])

# Layer-independent background input
bg_layer_independent = array([2000, 1850 ,2000, 1850, 2000, 1850, 2000, 1850])


def PDnet(NeuronGroup, stim, bg_type, w_ex, g, bg_freq, nsyn_type, thal, pfc):
    # P and D are the initials of the authors of the paper

    w_ex = w_ex*pA		   	# excitatory synaptic weight
    std_w_ex = 0.1*w_ex     # standard deviation weight

    pop = [] # Stores NeuronGroups, one for each population
    for r in range(0, 9):
        pop.append(NeuronGroup[nn_cum[r]:nn_cum[r+1]])

    ###########################################################################
    # Creating synapse connections
    ###########################################################################

    syn_model = '''
                w:amp			# synaptic weight
                '''

    # equations executed only when presynaptic spike occurs:
    # for excitatory connections
    pre_eq = '''
            I_post += w
            '''

    con = [] # Stores connections

    ###########################################################################
    # Connecting neurons
    ###########################################################################

    pre_index = [] # presynaptic neuron index
    post_index = [] # postsynaptic neuron index

    # c: column (presynaptic neurons)
    # r: row (postsynaptic neurons)
    for c in range(0, 8):
        for r in range(0, 8):

            if (nsyn_type==0):
                # number of synapses calculated with the original equation
                nsyn = int(log(1.0-table[r][c])/log(1.0 - (1.0/float(n_layer[c]*n_layer[r]))))
            elif (nsyn_type==1):
                # number of synapses calculated with the Taylor series approximation
                nsyn = int(n_layer[c]*n_layer[r]*table[r][c])
            pre_index = randint(n_layer[c], size=nsyn)
            post_index = randint(n_layer[r], size=nsyn)

            # assign weights to the synapses
            if nsyn<1:
                pass
            else:
                # Excitatory connections
                if (c % 2) == 0:
                    # Synaptic weight from L4e to L2/3e is doubled
                    if c == 2 and r == 0:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = '2.0*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    else:
                        con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                        con[-1].connect(i = pre_index, j = post_index)
                        con[-1].w = 'clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_ex + std_d_ex*randn(), 0.1*ms, d_ex*inf)'

                # Inhibitory connections
                else:
                    con.append(Synapses(pop[c], pop[r], model=syn_model, on_pre=pre_eq))
                    con[-1].connect(i = pre_index, j = post_index)
                    con[-1].w = '-g*clip((w_ex + std_w_ex*randn()),w_ex*0.0, w_ex*inf)'
                    con[-1].delay = 'clip(d_in + std_d_in*randn(), 0.1*ms, d_in*inf)'

    ###########################################################################
    # Setting background input values
    ###########################################################################

    # Background number per layer
    if bg_type == 0:
        # layer-specific:
        bg_layer = bg_layer_specific
    elif bg_type == 1:
        # layer-independent:
        bg_layer = bg_layer_independent

    ###########################################################################
    # Creating poissonian and DC-current background inputs
    ###########################################################################

    bg_in  = []
    if (stim==0):
        for r in range(0, 8):
            bg_in.append(PoissonInput(pop[r], 'I', bg_layer[r], bg_freq*Hz, weight=w_ex))

    # DC-current normalized by population
    if (stim == 1):
        for r in range(0, 8):
            NeuronGroup.Iext[pop[r]] = 0.3512*pA*bg_layer[r]

    ###########################################################################
    # Creating thalamic neurons as poissonian inputs
    ###########################################################################

    thal_con = []
    thal_input = []
    if thal=="ON":
        thal_input = PoissonGroup(n_layer[8], rates=120.0*Hz) #from PD paper: rates=15Hz
        for r in range(0,8):
            thal_con.append(Synapses(thal_input, pop[r], model=syn_model, on_pre=pre_eq))
            thal_con[-1].connect(p=table[r][8])
            thal_con[-1].w = 0.0

    ###########################################################################
    # Creating prefrontal neurons as poissonian inputs
    ###########################################################################

    pfc_con = []
    pfc_input = []
    if pfc=="ON":
        pfc_input = PoissonGroup(n_layer[9], rates=50.0*Hz) #from PD paper: rates=15Hz
        for r in range(0,8):
            pfc_con.append(Synapses(pfc_input, pop[r], model=syn_model, on_pre=pre_eq))
            pfc_con[-1].connect(p=table[r][9])
            pfc_con[-1].w = 0.0

    ###########################################################################
    # Creating spike monitors
    ###########################################################################

    smon_net = SpikeMonitor(NeuronGroup)

    return pop, con, bg_in, smon_net, thal_input ,thal_con, pfc_input, pfc_con

def runParams(tsim=1.0, bg_type=0, stim=0, w_ex=87.8, g=4.0, bg_freq=8.0, nsyn_type=0, thal='OFF', pfc='OFF', filename=None):

    ###########################################################################
    # Simulation parameters
    ###########################################################################
    defaultclock.dt = 0.1*ms    # timestep of numerical integration method

    # neuron model
    eqs = LIFmodel
    reset = resetLIF
    tau_m, tau_ref, Cm, v_r, v_th = LIFparams()

    ###########################################################################
    # Creating neurons
    ###########################################################################
    neurons = NeuronGroup(N, eqs, threshold='v>v_th', reset=reset, \
                            method='linear', refractory=tau_ref)

    # seting initial values for membrane potential and currents
    neurons.v = '-58.0*mV + 10.0*mV*randn()'
    neurons.I = 0.0*pA      # initial value for synaptic currents
    neurons.Iext = 0.0*pA   # constant external current

    pop, con, bg_in, smon_net, thal_input, thal_con, pfc_input, pfc_con = PDnet(neurons, stim, \
                                            bg_type, w_ex, g, bg_freq, nsyn_type, thal, pfc)

    ###########################################################################
    # Running the simulation
    ###########################################################################
    net = Network(collect())

    if (thal == 'OFF' and pfc == 'OFF'):
        net.add(neurons,pop, con, bg_in)    # Adding objects to the simulation
        net.run(tsim*second, report='stdout')

    elif (thal == 'ON'):
        w_thal = w_ex*pA            # excitatory synaptic weight from thalamus
        std_w_thal = w_thal*0.1     # standard deviation weigth
        net.add(neurons,pop, con, bg_in, thal_input, thal_con)    # Adding objects to the simulation

        for repeat in range(0,int(tsim)):
            net.run(0.7*second,report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Adding thalamic input
            for r in range(0,8):
                thal_con[r].w = 'clip((w_thal + std_w_thal*randn()),w_thal*0.0, w_thal*inf)'
            net.run(0.01*second, report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Removing thalamic input
            for r in range(0,8):
                thal_con[r].w = 0
            net.run(0.29*second, report='stdout')
            gc.collect()    #garbage collector to clean memory

    elif (pfc == 'ON'):
        w_pfc = w_ex*pA            # excitatory synaptic weight from prefrontal cortex
        std_w_pfc = w_pfc*0.1     # standard deviation weigth
        net.add(neurons,pop, con, bg_in, pfc_input, pfc_con)    # Adding objects to the simulation

        for repeat in range(0,int(tsim)):
            net.run(0.5*second,report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Adding thalamic input
            for r in range(0,8):
                pfc_con[r].w = 'clip((w_pfc + std_w_pfc*randn()),w_pfc*0.0, w_pfc*inf)'
            net.run(0.3*second, report='stdout')
            gc.collect()    #garbage collector to clean memory

            # Removing thalamic input
            for r in range(0,8):
                pfc_con[r].w = 0
            net.run(0.2*second, report='stdout')
            gc.collect()    #garbage collector to clean memory


    ###########################################################################
    # Saving raster plot
    ###########################################################################
    savetxt(filename, c_[smon_net.i,smon_net.t/ms],fmt="%i %.2f")

def rasterplot_pfc(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample):
  ###############################################################################
  # Raster plot
  ###############################################################################
  plt.figure(figsize=(7.5,10))
  plt.gca().set_yticklabels([])
  acum_index = 0

  # graphs color codes: different colors for different layers
  dotsize = 2.5
  dotcolor = np.array([[0.0, 0.0, 255.0],
                      [102.0, 178.0, 255.0],
                      [255.0, 128.0, 0.0],
                      [255.0, 178.0, 102.0],
                      [0.0,   128.0, 0.0],
                      [153.0, 255.0, 153.0],
                      [255.0, 0.0,   0.0],
                      [255.0, 153.0, 153.0]])/255.0

  for i in range(len(lname)):
      index_start = l_bins[i]
      index_end = l_bins[i]+int(psample*n_layer_plot[i+1])

      x = data.t[data.i.isin(range(index_start,index_end))]
      y = data.i[data.i.isin(range(index_start,index_end))] + acum_index - index_start

      plt.plot(x,y,'.',markersize=dotsize,color=dotcolor[i])

      # layers labels
      xpos = tsim*1000.0-520
      ypos = acum_index + (index_end-index_start)/2.0
      plt.text(xpos,ypos,lname[i],horizontalalignment='center', fontsize=12, rotation=30)


      acum_index = acum_index + (index_end-index_start)

  plt.xlim(tsim*1000.0-510,tsim*1000.0-200)
  plt.ylim(0,acum_index)
  plt.xlabel('time [ms]')
  plt.ylabel(' ')
  plt.gca().invert_yaxis()
  plt.show()

def stepplot_pfc(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample):
  spk_count = []
  bins = np.arange(0,tsim*1000.0+.5,.5)

  for i in range(len(lname)):
      index_start = l_bins[i]
      index_end = l_bins[i]+int(psample*n_layer_plot[i+1])
      count, division = np.histogram(data.t[data.i.isin(np.arange(index_start,index_end))],bins=bins)
      #count = sum(np.split(count,int(tsim)))/float(tsim)
      spk_count.append([count])

  measures_layer['spk_count'] = spk_count

  for i in range(0,len(lname),2):
      plt.subplot2grid((4,3),(int(i/2),1),colspan=2)
      plt.step(np.arange(-10,300,0.5),measures_layer.spk_count[i][0][980:1600], label=lname[i])
      plt.step(np.arange(-10,300,0.5),measures_layer.spk_count[i+1][0][980:1600], label=lname[i+1])
      plt.legend()
      if i != 6:
          ax = plt.gca()
          ax.xaxis.set_visible(False)
      else:
          ax = plt.gca()
          ax.yaxis.set_visible(True)
      plt.ylim(0,13)

  plt.show()

%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 1 # DC inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, pfc="ON", filename=filename(g, bg, '_pfc'))
gc.collect() #garbage collector to clean memory

spk_neuron, measures_layer, data = preprocess(tsim, filename(g, bg, '_pfc'), lname, n_layer_plot, l_bins, N, psample, n_sample)
rasterplot_pfc(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)
stepplot_pfc(spk_neuron, tsim, lname, n_layer_plot, l_bins, N, psample, n_sample)

# Graded Assignment: In Search for a Good Model

*Note*. This is a take-home assignment. You are not required to work on it during the in-class tutorial.

After running through the tutorial, we have built a spike network from sractch and studied its spontaneous activities, tested its robustness, and investigated the effects of transient stimuli. Standing upon the giant's shoulder, one wonders: could this model be any better?

<font color="red">**To Think A1**</font>: To evaluate the goodness of a neural model, what do you think that are the desiderata? List at least 3 criteria. (3 points)

We defined lots of parameters and equations, and all of them might be subject to optimization. In the rest of the assignment, we will reconsider several parameters in this network.

## The Synapse and Neuron Number

We have two aforementioned equations to calculate the synapse number between two neuronal populations:
$$
K = \ln(C_a) / \ln(1 - (1 - \frac{1}{N^{pre}N^{post}}))
$$
and
$$
K = \frac{C_a}{N^{pre}N^{post}}
$$

The latter is the Taylor series approximation of the first equation. Using different equations will change the synapse number hence the network structure. We will simulate two networks created with different synapse number equations.

<font color="green">**To Do A1**</font>: Compare the synapse numbers calculated by either equation. Display the changes in synapse numbers and report your finding (2 point).

In [ ]:
##YOUR CODE##

Now we are testing whether using different equations alters our network in its behaviour. Let's run the simulations. You may notice the simulations are very RAM-taxing process, does the new equation implementation make the situation better?

#### No Synapse Number Approximation (Default Setting)

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 0 # Poissonian inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, nsyn_type=0, filename=filename(g, bg, '_noapprox'))
gc.collect() #garbage collector to clean memory

#### Approximated Synapse Number

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 0 # Poissonian inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, nsyn_type=1, filename=filename(g, bg, '_approx'))
gc.collect() #garbage collector to clean memory

### Kolmogorov-Smirnov Test

We utilize the non-parametric Kolmogorov-Smirnov statistics to quantify the difference in the firing rates and coefficient of variance.

In [ ]:
#@title KS Test Function
%matplotlib inline
from scipy.stats import ks_2samp

def ks_test(tsim):

    ###############################################################################
    # Filenames
    ###############################################################################
    filename = (['../data/data_raster_g4.0_bgrate8.0_approx.dat',\
                '../data/data_raster_g4.0_bgrate8.0_noapprox.dat'])

    ###############################################################################
    # Parameters
    ###############################################################################

    n = 77169 # total number of neurons

    # cortical layer labels: e for excitatory; i for inhibitory
    lname = ['L23e', 'L23i', 'L4e', 'L4i', 'L5e', 'L5i','L6e', 'L6i']

    # number of neurons by layer
    n_layer = [0, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948];
    l_bins = np.cumsum(n_layer) # cumulative number of neurons by layer

    # dataframe to store CV's and firing rates
    measures = pd.DataFrame()

    for k in range(0,2):

        # loading data
        data = pd.read_csv(filename[k], sep=" ", header=None, names=['i','t'])

        # grouping spiking times for each neuron
        keys,values = data.sort_values(['i','t']).values.T
        ukeys,index=np.unique(keys,True)
        arrays=np.split(values,index[1:])
        spk_neuron = pd.DataFrame({'i':ukeys,'t':[list(a) for a in arrays]})

        # creating a flag to identify cortical layers
        spk_neuron['layer'] = pd.cut(spk_neuron['i'], l_bins, labels=lname, right=False)
        data['layer'] = pd.cut(data['i'], l_bins, labels=lname, right=False)

        # cleaning variables
        del keys, values, ukeys, index, arrays

        ###############################################################################
        # Interspike intervals + coefficient of variation
        ###############################################################################
        # interspike intervals
        isi = []
        isi = [np.diff(spk_neuron.t[i]) for i in range(len(spk_neuron))]

        # coefficient of variation
        aux = []
        aux = [np.std(isi[i])/np.mean(isi[i]) if len(isi[i])>1 else np.nan\
                for i in range(len(spk_neuron))]

        cv = np.zeros(n)*np.nan
        cv[spk_neuron.i.astype(int)] = aux

        if k == 1:
            measures['cv_noaprox'] = cv
        else:
            measures['cv_aprox'] = cv

        ###############################################################################
        # Firing rates
        ###############################################################################
        aux = []
        aux = [float(len(spk_neuron.t[i]))/tsim for i in range(len(spk_neuron))]

        freq = np.zeros(n)
        freq[spk_neuron.i.astype(int)] = aux

        if k == 1:
            measures['f_noaprox'] = freq
        else:
            measures['f_aprox'] = freq

        # cleaning variables
        del data, spk_neuron

    measures['layer'] = pd.cut(measures.index, l_bins, labels=lname, right=False)

    # arrays to store the p-value from Kolmogorov-Smirnov test
    ks_cv = np.zeros(8)
    ks_f = np.zeros(8)

    p_cv = np.zeros(8)
    p_f = np.zeros(8)

    # create figures to plot cummulative histograms
    fig1, ax1 = plt.subplots(nrows=2, ncols=4, figsize=(10,6))
    fig2, ax2 = plt.subplots(nrows=2, ncols=4, figsize=(10,6))

    for i in range(0,8):

        # grid positions for the graph
        xgrid = int(i%2)
        ygrid = int(i/2)

        ################################################################
        # CV's distribution comparison
        ################################################################
        plt.figure(1)
        plt.subplot2grid((2,4),(xgrid,ygrid))
        plt.title(lname[i])

        # excluding NAN values
        aux1 = measures.groupby('layer')['cv_aprox'].get_group(lname[i])
        aux1 = aux1[~np.isnan(aux1)]

        aux2 = measures.groupby('layer')['cv_noaprox'].get_group(lname[i])
        aux2 = aux2[~np.isnan(aux2)]

        # CV's histograms
        cv1,edges1 = np.histogram(aux1,bins='sqrt',range=(0,2.0))
        cv2,edges2 = np.histogram(aux2,bins=np.linspace(0,2.0,len(edges1)))

        # cummulated histograms
        cv1 = np.cumsum(cv1)
        cv2 = np.cumsum(cv2)

        # normalizing cummulated histograms
        cv1 = cv1/float(max(cv1))
        cv2 = cv2/float(max(cv2))

        # plot data
        plt.plot(edges1[:-1],cv1,'--', label='Approximated')
        plt.plot(edges2[:-1],cv2,label='Not Approximated')
        if (i==0): plt.legend(loc=4)
        if (i>1): plt.yticks([])
        if (i%2==0): plt.xticks([])
        plt.autoscale(enable=True, axis='x', tight=True)

        # Kolmogorov-Smirnov statistical test
        ks_cv[i], p_cv[i] = ks_2samp(cv1, cv2)

        del aux1, aux2, cv1, cv2

        ################################################################
        # firing rate distribution comparison
        ################################################################
        plt.figure(2)
        plt.subplot2grid((2,4),(xgrid,ygrid))
        plt.title(lname[i])

        # excluding NAN values
        aux1 = measures.groupby('layer')['f_aprox'].get_group(lname[i])
        aux1 = aux1[~np.isnan(aux1)]

        aux2 = measures.groupby('layer')['f_noaprox'].get_group(lname[i])
        aux2 = aux2[~np.isnan(aux2)]

        # firing rate histograms
        f1,edges1 = np.histogram(aux1,bins='sqrt',range=(0,max(aux1)))
        f2,edges2 = np.histogram(aux2,bins=np.linspace(0,max(aux1),len(edges1)))

        # cummulated histograms
        f1 = np.cumsum(f1)
        f2 = np.cumsum(f2)

        # normalizing cummulated histograms
        f1 = f1/float(max(f1))
        f2 = f2/float(max(f2))

        # plot data
        plt.plot(edges1[:-1],f1,'--', label='Approximated')
        plt.plot(edges1[:-1],f2,label='Not Approximated')
        if (i==0): plt.legend(loc=4)
        if (i>1): plt.yticks([])
        plt.autoscale(enable=True, axis='x', tight=True)

        # Kolmogorov-Smirnov statistical test
        ks_f[i], p_f[i] = ks_2samp(f1, f2)

        del aux1, aux2, f1, f2

    # formating figures:ks_2samp
    plt.figure(1)
    fig1.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.xlabel('CV', fontsize=12, labelpad=10)
    plt.ylabel('cumulative distribution',fontsize=12, labelpad=12)
    plt.tight_layout()

    plt.figure(2)
    fig2.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.xlabel('firing rate [Hz]', fontsize=12, labelpad=10)
    plt.ylabel('cumulative distribution',fontsize=12, labelpad=12)
    plt.tight_layout()
    plt.show()

    return ks_cv, ks_f, p_cv, p_f

def ks_test_n_neuron(tsim, scale=0.95):

    ###############################################################################
    # Filenames
    ###############################################################################
    filename = (['../data/data_raster_g4.0_bgrate8.0_less.dat',\
                '../data/data_raster_g4.0_bgrate8.0default.dat'])

    ###############################################################################
    # Parameters
    ###############################################################################

    # cortical layer labels: e for excitatory; i for inhibitory
    lname = ['L23e', 'L23i', 'L4e', 'L4i', 'L5e', 'L5i','L6e', 'L6i']

    # dataframe to store CV's and firing rates
    measures_less = pd.DataFrame()
    measures_default = pd.DataFrame()

    for k in range(0,2):

      if k == 0:
        n_layer = np.multiply([0, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948], scale).astype(int).tolist()
        n = sum(n_layer)
        l_bins = np.cumsum(n_layer)
      elif k == 1:
        n_layer = [0, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948]
        n = sum(n_layer)
        l_bins = np.cumsum(n_layer)

      # loading data
      data = pd.read_csv(filename[k], sep=" ", header=None, names=['i','t'])

      # grouping spiking times for each neuron
      keys,values = data.sort_values(['i','t']).values.T
      ukeys,index=np.unique(keys,True)
      arrays=np.split(values,index[1:])
      spk_neuron = pd.DataFrame({'i':ukeys,'t':[list(a) for a in arrays]})

      # creating a flag to identify cortical layers
      spk_neuron['layer'] = pd.cut(spk_neuron['i'], l_bins, labels=lname, right=False)
      data['layer'] = pd.cut(data['i'], l_bins, labels=lname, right=False)

      # cleaning variables
      del keys, values, ukeys, index, arrays

      print(f"k={k}, spk_neuron.shape={spk_neuron.shape}, n={n}")

      ###############################################################################
      # Interspike intervals + coefficient of variation
      ###############################################################################
      # interspike intervals
      isi = []
      isi = [np.diff(spk_neuron.t[i]) for i in range(len(spk_neuron))]

      # coefficient of variation
      aux = []
      aux = [np.std(isi[i])/np.mean(isi[i]) if len(isi[i])>1 else np.nan\
              for i in range(len(spk_neuron))]

      cv = np.zeros(n)*np.nan
      print(f"cv.shape={cv.shape}, aux.shape={len(aux)}")
      print(f"spk_neuron.i.shape={spk_neuron.i.shape}")
      cv[spk_neuron.i.astype(int)] = aux

      if k == 1:
          measures_default['cv_default'] = cv
      else:
          measures_less['cv_less'] = cv

      ###############################################################################
      # Firing rates
      ###############################################################################
      aux = []
      aux = [float(len(spk_neuron.t[i]))/tsim for i in range(len(spk_neuron))]

      freq = np.zeros(n)
      freq[spk_neuron.i.astype(int)] = aux

      if k == 1:
          measures_default['f_default'] = freq
      else:
          measures_less['f_less'] = freq

      # cleaning variables
      del data, spk_neuron

    measures_default['layer'] = pd.cut(measures_default.index, l_bins, labels=lname, right=False)
    measures_less['layer'] = pd.cut(measures_less.index, l_bins, labels=lname, right=False)

    # arrays to store the p-value from Kolmogorov-Smirnov test
    ks_cv = np.zeros(8)
    ks_f = np.zeros(8)

    p_cv = np.zeros(8)
    p_f = np.zeros(8)

    # create figures to plot cummulative histograms
    fig1, ax1 = plt.subplots(nrows=2, ncols=4, figsize=(10,6))
    fig2, ax2 = plt.subplots(nrows=2, ncols=4, figsize=(10,6))

    for i in range(0, 7):

        # grid positions for the graph
        xgrid = int(i%2)
        ygrid = int(i/2)

        ################################################################
        # CV's distribution comparison
        ################################################################
        plt.figure(1)
        plt.subplot2grid((2,4),(xgrid,ygrid))
        plt.title(lname[i])

        # excluding NAN values
        aux1 = measures_less.groupby('layer')['cv_less'].get_group(lname[i])
        aux1 = aux1[~np.isnan(aux1)]

        aux2 = measures_default.groupby('layer')['cv_default'].get_group(lname[i])
        aux2 = aux2[~np.isnan(aux2)]

        # CV's histograms
        cv1,edges1 = np.histogram(aux1,bins='sqrt',range=(0,2.0))
        cv2,edges2 = np.histogram(aux2,bins=np.linspace(0,2.0,len(edges1)))

        # cummulated histograms
        cv1 = np.cumsum(cv1)
        cv2 = np.cumsum(cv2)

        # normalizing cummulated histograms
        cv1 = cv1/float(max(cv1))
        cv2 = cv2/float(max(cv2))

        # plot data
        plt.plot(edges1[:-1],cv1,'--', label=f'{scale * 100}% neurons')
        plt.plot(edges2[:-1],cv2,label='All neurons')
        if (i==0): plt.legend(loc=4)
        if (i>1): plt.yticks([])
        if (i%2==0): plt.xticks([])
        plt.autoscale(enable=True, axis='x', tight=True)

        # Kolmogorov-Smirnov statistical test
        ks_cv[i], p_cv[i] = ks_2samp(cv1, cv2)

        del aux1, aux2, cv1, cv2

        ################################################################
        # firing rate distribution comparison
        ################################################################
        plt.figure(2)
        plt.subplot2grid((2,4),(xgrid,ygrid))
        plt.title(lname[i])

        # excluding NAN values
        aux1 = measures_less.groupby('layer')['f_less'].get_group(lname[i])
        aux1 = aux1[~np.isnan(aux1)]

        aux2 = measures_default.groupby('layer')['f_default'].get_group(lname[i])
        aux2 = aux2[~np.isnan(aux2)]

        # firing rate histograms
        f1,edges1 = np.histogram(aux1,bins='sqrt',range=(0,max(aux1)))
        f2,edges2 = np.histogram(aux2,bins=np.linspace(0,max(aux1),len(edges1)))

        # cummulated histograms
        f1 = np.cumsum(f1)
        f2 = np.cumsum(f2)

        # normalizing cummulated histograms
        f1 = f1/float(max(f1))
        f2 = f2/float(max(f2))

        # plot data
        plt.plot(edges1[:-1],f1,'--', label=f'{scale * 100}% neurons')
        plt.plot(edges1[:-1],f2,label='All neurons')
        if (i==0): plt.legend(loc=4)
        if (i>1): plt.yticks([])
        plt.autoscale(enable=True, axis='x', tight=True)

        # Kolmogorov-Smirnov statistical test
        ks_f[i], p_f[i] = ks_2samp(f1, f2)

        del aux1, aux2, f1, f2

    # formating figures:ks_2samp
    plt.figure(1)
    fig1.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.xlabel('CV', fontsize=12, labelpad=10)
    plt.ylabel('cumulative distribution',fontsize=12, labelpad=12)
    plt.tight_layout()

    plt.figure(2)
    fig2.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.xlabel('firing rate [Hz]', fontsize=12, labelpad=10)
    plt.ylabel('cumulative distribution',fontsize=12, labelpad=12)
    plt.tight_layout()
    plt.show()

    return ks_cv, ks_f, p_cv, p_f

In [ ]:
ks_cv, ks_f, p_cv, p_f = ks_test(tsim)

In [ ]:
# check the KS statistics
print(f"ks_cv={ks_cv}, ks_f={ks_f}")
print(f"p_cv={p_cv}, p_f={p_f}")

#### Less Neurons in the Network

If we only use 95% of the original number of neurons, what would change? Let's run a simulation again and do a similar KS test and see what's happened.

In [ ]:
n_layer = np.multiply([20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948, 902], 0.95).astype(int).tolist()
N = sum(n_layer[:-1])
nn_cum = [0]
nn_cum.extend(cumsum(n_layer))

%cd /content/drive/MyDrive/spike_network/code

g = 4.0 # default value for inhibitory weight balance
bg = 8.0 # default value for background rate

bg_type = 0 # layer-specific inputs
stim = 0 # Poissonian inputs
tsim = 1 # time for simulation: 1 sec
runParams(tsim=tsim, bg_type=bg_type, stim=stim, nsyn_type=0, filename=filename(g, bg, '_less'))
gc.collect() #garbage collector to clean memory

In [ ]:
# revert important parameters
n_layer = [20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948, 902]
N = sum(n_layer[:-1])
nn_cum = [0]
nn_cum.extend(cumsum(n_layer))

In [ ]:
ks_cv, ks_f, p_cv, p_f = ks_test_n_neuron(tsim, scale=0.95)

In [ ]:
# check the KS statistics
print(f"ks_cv={ks_cv}, ks_f={ks_f}")
print(f"p_cv={p_cv}, p_f={p_f}")

<font color="green">**To Do A2**</font> (1 point):
1. Try to simulate using only 80% neurons and run a KS test. Are the changes more significant? Be careful: remember to revert the neuron numbers after your experiment since they are very important parameters.

In [ ]:
##YOUR CODE##

<font color="red">**To Think A2**</font>:
So far we have tinkered on two structural factors to "slim" down the network. Do those methods make the model better or worse in any sense? What do they help you understand more about the model (3 points)?

## The Asychronous Irregular (AI) Firing regime

*Note*. Running this section may take >1 hour.

The low-rate AI firing regime has been considered to be the ground state of cortical activity (Amit and Brunel 1997). And it might be related to the inhibitory weights and background rates. We can test it with our network by trying several inhibitory weights and background rates.

To this end, we need to define a measure determining how AI a neuronal population is. We define AIness%, the percentage of populations with the firing rate <30 Hz, irregularity between 0.7 and 1.2, and synchrony <8, as a function of the background rate and the relative inhibitory synaptic strength.

*Note*. We made some practical changes to this module. We deminished the number of simulations we need to do by testing less inhibitory weight and background rates values.

In [ ]:
#@title Plotting Functions: AI Firing Regime

%matplotlib inline

def datafig6(tsim):

    g = np.arange(2.0,10.5,2);        # relative inh. synaptic strength values
    bg_rate = np.arange(2.0,15.5,3);  # background rate values

    ###############################################################################
    # Parameters
    ###############################################################################

    # cortical layer labels: e for excitatory; i for inhibitory
    lname = ['L23e', 'L23i', 'L4e', 'L4i', 'L5e', 'L5i','L6e', 'L6i']

    # number of neurons by layer
    n_layer = [0, 20683, 5834, 21915, 5479, 4850, 1065, 14395, 2948];
    l_bins = np.cumsum(n_layer) # cumulative number of neurons by layer
    N = np.sum(n_layer)         # total number of neurons

    # creating dataframes to store measures
    ainess = np.zeros((len(bg_rate),len(g))) # AIness index
    freq_g4 = pd.DataFrame(index=lname)      # firing rates for g = 4.0
    freq_bg8 = pd.DataFrame(index=lname)     # firing rates for bg = 8.0 Hz

    for row in range(len(g)):
        for col in range(len(bg_rate)):

            # loading data to a DataFrame structure
            filename = '../data/data_raster_g'+str(g[row])+'_bgrate'+str(bg_rate[col])+'.dat'
            data = pd.read_csv(filename, sep=" ", header=None, names=['i','t'])

            # grouping spiking times for each neuron
            keys,values = data.sort_values(['i','t']).values.T
            ukeys,index=np.unique(keys,True)
            arrays=np.split(values,index[1:])

            spk_neuron = pd.DataFrame({'i':range(0,N),'t':[[]]*N})
            # DEPRECATED: List of arrays cannot be directly assigned to dataframe
            # spk_neuron.iloc[ukeys.astype(int),1] = arrays
            for neuron_index, spike_times in zip(ukeys.astype(int), arrays):
                spk_neuron.at[neuron_index, 't'] = spike_times.tolist()

            # creating a flag to identify cortical layers
            spk_neuron['layer'] = pd.cut(spk_neuron['i'], l_bins, labels=lname, right=False)
            data['layer'] = pd.cut(data['i'], l_bins, labels=lname, right=False)

            # sampling data
            n_sample = 1000 # number of neurons by layer for sampled measures
            spk_neuron = spk_neuron.groupby(['layer']).apply(lambda x: x.sample(n=n_sample))

            # Reset the index to remove the ambiguity
            spk_neuron = spk_neuron.reset_index(drop=True)

            # measures DataFrame:
            measures_layer = pd.DataFrame(index=lname)

            # cleaning variables
            del keys, values, ukeys, index, arrays

            ########################################################################
            # Mean firing rates
            ########################################################################
            freq = []
            freq = [float(len(spk_neuron.t[i]))/tsim for i in range(len(spk_neuron))]
            spk_neuron['f'] = freq

            measures_layer['f'] = spk_neuron.groupby(['layer'])['f'].mean()

            if g[row]==4.0: freq_g4[bg_rate[col]] = measures_layer.f
            if bg_rate[col] == 8.0: freq_bg8[g[row]] = measures_layer.f

            ########################################################################
            # Interspike intervals + coefficient of variation
            ########################################################################
            # interspike intervals
            isi = []
            isi = [np.diff(spk_neuron.t[i]) for i in range(len(spk_neuron))]

            # coefficient of variation
            cv = []
            cv = [np.std(isi[i])/np.mean(isi[i]) if len(isi[i])>1 else np.nan \
                    for i in range(len(spk_neuron))]
            spk_neuron['cv'] = cv

            measures_layer['cv'] = spk_neuron.groupby(['layer'])['cv'].mean()

            ########################################################################
            # Synchrony index
            ########################################################################
            sync = []
            bins = np.arange(0,tsim*1000.0+3.0,3)

            for i in range(len(lname)):
                index_sample = spk_neuron.i[spk_neuron.layer.isin([lname[i]])]
                count, division = np.histogram(data.t[data.i.isin(index_sample)],bins=bins)

                # removing first 100 ms of simulation
                sync.append(np.var(count[166:])/np.mean(count[166:]))

            measures_layer['sync'] = sync

            ########################################################################
            # AIness measure: f<30Hz & 0.7<=cv<1.2 & sync_index <= 8
            ########################################################################
            measures_layer['AI'] = (measures_layer.f<30)&(measures_layer.sync<=8)&\
                                    (measures_layer.cv>=0.7)&(measures_layer.cv<1.2)

            # % of layers in the AIness range
            ainess[col][row] = 100*sum(measures_layer.AI)/8.0

            # cleaning variables
            del measures_layer, spk_neuron, data

    # saving files
    np.savetxt('../data/ainess.dat', ainess)
    freq_g4.to_csv('../data/freq_g4.csv')
    freq_bg8.to_csv('../data/freq_bg8.csv')


def createfig6():

    # loading data
    data = pd.read_csv('../data/ainess.dat', header = None, sep=' ')
    freq_g4 = pd.read_csv('../data/freq_g4.csv', index_col=0); freq_g4 = freq_g4.T
    freq_bg8 = pd.read_csv('../data/freq_bg8.csv', index_col=0); freq_bg8 = freq_bg8.T

    bg = np.arange(2.0,15.5,3)
    g = np.arange(2,10.5,2)

    plotstyle = ['-.','-^','-*','-s']

    # figure size for the graphs
    fig = plt.figure(figsize=(8,8))
    gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 0.2], height_ratios=[3, 1])

    # fig. 8A: firing rates for fixed parameter g = 4.0
    ax1 = plt.subplot(gs[0,0])
    plt.plot(freq_g4.L23e,bg, plotstyle[0])
    plt.plot(freq_g4.L4e,bg, plotstyle[1])
    plt.plot(freq_g4.L5e,bg, plotstyle[2])
    plt.plot(freq_g4.L6e,bg, plotstyle[3])
    plt.ylim([2,15])
    plt.gca().invert_xaxis()
    plt.xlabel('firing rate [Hz]')
    plt.xlim(16, 0)

    # fig. 8C: firing rates for fixed parameter background rate = 8.0Hz
    ax2 = plt.subplot(gs[1,1])
    freq_bg8.plot(y=['L23e','L4e','L5e','L6e'], ax=ax2, style=plotstyle)
    plt.gca().invert_yaxis()
    plt.ylabel('firing rate [Hz]')

    # fig. 8B: contour plot for %AIness
    data = data.sort_index(ascending=False)
    bg = np.arange(15.5,2.0,-3)

    ax3 = plt.subplot(gs[0,1])
    levels1 = np.linspace(0, 100, num=5, endpoint='True')
    CS1 = plt.contour(g, bg, data, levels=levels1, colors='k', linestyles='dashed')
    plt.clabel(CS1, inline=1, fontsize=10, fmt='%1.0f')
    CS2 = plt.imshow(data, extent=[2,10,2,15], cmap='jet', interpolation='gaussian', aspect='auto')

    plt.xlabel('relative inh. synaptic strength')
    plt.ylabel('background rate [Hz]')

    # colorbar of contour plot
    ax4 = plt.subplot(gs[0,2])
    plt.colorbar(CS2, cax=ax4, ticks=np.arange(0,101,50))
    plt.title('%AIness')

    plt.tight_layout()
    plt.show()

In [ ]:
%cd /content/drive/MyDrive/spike_network/code

g_values = np.arange(2.0, 11.0, 2);  # relative inh. synaptic strength values
bg_values = np.arange(2.0, 15.5, 3);  # background rate values

bg_type = 0
stim = 0
tsim = 1

for g in g_values:
    for bg in bg_values:
        runParams(tsim=tsim, bg_type=bg_type, stim=stim, g=g, \
        bg_freq=bg, filename=filename(g, bg))
        gc.collect()    #garbage collector to clean memory

In [ ]:
datafig6(tsim)
createfig6()

<font color="red">**To Think A3**</font>

1. To achieve sufficient AIness in sponetaneous activity, how should be inhibitory weights and background rates be tuned? What knowledge about the network mechanism can you gain from this experiment? (2 points)

2. What can you do with this network? Provide a research question and desribe how you going to tackle it with this network. (2 points)

3. What are you still not satisfied with this network? Desribe one limitation of this network. (1 point)